# Prepare Input

This notebook create xarray dataset to be used as the input to the B2BNet DL model.

## Participants

In [1]:
# Setups
import numpy as np
import mne
from pathlib import Path
import re
import xarray as xr
import pandas as pd

eeg_path = 'data/EEG/participants/'

# eeg channel names (second dimension in dataset)
name = 'sub-01_ses-01_task-baseline1_proc-clean_epo.fif'
epochs = mne.read_epochs(eeg_path+name, verbose=0)
ch_names = epochs.ch_names

# collector dataset
ds = xr.Dataset()

In [3]:
# Find relaxation session number (The dataset only include the relaxation procedure)
data = pd.read_csv('codes/B2BC/data/behavioral/behavioral_data.csv')
relax_ses = data.query('procedure == "relaxation"')[['bids_id', 'session']].reset_index(drop=True)

# Change bids id to match file names
relax_ses['bids_id'] = relax_ses['bids_id'].astype('int').astype('str').str.zfill(2)

# List eeg files names (for relaxation sessions)
eegfile_names = []
for i in range(len(relax_ses)):
    id = relax_ses.loc[i, 'bids_id']
    ses = str(relax_ses.loc[i, 'session'])
    eegfile_names.append(f'sub-{id}_ses-01_task-induction{ses}_proc-clean_epo')

# remove the last file name (we did not preprocess the eeg data from this session because of the program bug)
eegfile_names = eegfile_names[:-1]

In [4]:
# open relaxation eeg files
for name in eegfile_names:
    fpath = eeg_path + name + '.fif'
    sub, task = re.search('sub-(.*)_ses-01_task-(.*)_proc-clean_epo', name).groups()
    epochs = mne.read_epochs(fpath, verbose=0)
    epochs.resample(128)
    data = np.hstack(epochs.get_data())
    
    # create a xarray
    da = xr.DataArray(
    data=data,
    dims=["channels", "time"],
    coords=dict(
        ch=(["channels"], ch_names),
        time=np.arange(0, data.shape[1]),
    ))
    
    ds = ds.update({f'sub_{sub}': da})

## hypnotist

In [19]:
# open data 
raw1 = mne.io.read_raw_brainvision('data/hypnotist/Relaxation_whole.vhdr', verbose=0)

/var/folders/pn/qy0qc5tx7t7_97xjmy6pvksc0000gn/T/ipykernel_10830/542632945.py:2: RuntimeWarning: No coordinate information found for channels ['EMG2', 'EMG3', 'ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw1 = mne.io.read_raw_brainvision('data/hypnotist/Relaxation_whole.vhdr', verbose=0)
/var/folders/pn/qy0qc5tx7t7_97xjmy6pvksc0000gn/T/ipykernel_10830/542632945.py:2: RuntimeWarning: Not setting positions of 3 misc channels found in montage:
['EMG2', 'EMG3', 'ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw1 = mne.io.read_raw_brainvision('data/hypnotist/Relaxation_whole.vhdr', verbose=0)
/var/folders/pn/qy0qc5tx7t7_97xjmy6pvksc0000gn/T/ipykernel_10830/542632945.py:3: RuntimeWarning: No coordinate information found for channels ['EMG2', 'EMG3', 'ECG']. Setting channel types to misc. To avoid this 